In [69]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.rcParams.update({'font.size':14})

# Combining datasets with `pandas`

## Overview

```{admonition} Questions
:class: questions
- How do I work with multiple datasets?
- How do I match tables?
- How do I save a table?
```

```{admonition} Objectives
:class: objectives
- Combine data from different files into one dataframe
- Save the combined data to a file
```

This week we're going to look at how to combine different datasets using `pandas`.

So far we've been lucky that the data we've needed for our analyses has all been in a single file. But what about if you have data from different sources? How do you combine your different sources into one dataframe?

We're going to be using data on a sample of Cepheid variables. This data is all real astronomical research data. We'll be using information from the [European Space Agency's *Gaia* mission](https://sci.esa.int/web/gaia), [NASA IPAC](https://www.ipac.caltech.edu) along with some of my own data from ground-based telescopes. 

## Reading in the data

The first file we'll be working is [optical_data.csv](./data/optical_data.csv), which contains [photometric measurements](https://en.wikipedia.org/wiki/Photometry_(astronomy)) of a sample of Cepheids in our own galaxy. First, we need to read the data into a dataframe, and take a look at it to see what's there.

In [2]:
optical_df = pd.read_csv('./data/optical_data.csv')

In [3]:
optical_df.head()

,Star_ID,logP,mag_B,err_B,mag_V,err_V,mag_I,err_I
0,RT Aur,0.571,6.120,0.017,5.487,0.011,4.822,0.006
1,QZ Nor,0.578,9.782,0.007,8.875,0.004,7.871,0.003
2,SU Cyg,0.585,7.493,0.015,6.890,0.011,6.208,0.007
3,Y Lac,0.636,9.921,0.016,9.163,0.011,8.312,0.007
4,T Vul,0.647,6.444,0.014,5.772,0.009,5.087,0.006


`````{admonition} .head()
:class: myinfo
In the cell above I've used
```py
optical_df.head()
```
to display only the first 5 rows of the dataframe rather than the whole thing. You could also use
```py
optical_df[:5]
```
to get the same output.

You can change the number of rows that `head()` displays by passing the number of rows as an argument, e.g.
```py
optical_df.head(10)
```

`````

The data in this file is:
* `Star_ID` - the name of the star (these ones are nearby so they have actual names)
* `logP` - $\log_{10} P$, where $P$ is the period of the variable star (in days)
* `mag_X` - the apparent magnitude of the star in the `B`, `V` and `I` bands
* `err_X` - the uncertainty on the apparent magnitude in each band.

Given that I'm not displaying the whole dataframe, it's worth looking at some of it's **attributes** so we know how much data there is. `shape` is an attribute of all dataframes, so that's a good place to start.

In [4]:
optical_df.shape

(59, 8)

This tells us that `optical_df` has 8 columns and 59 rows.

Dataframes also have **methods** associated with them (methods were covered in the C3 worksheet if you want a refresher). We've already used one method today - `head()`. Another useful method to use here is `describe()`:

In [5]:
optical_df.describe()

,logP,mag_B,err_B,mag_V,err_V,mag_I,err_I
count,59.000000,58.000000,58.000000,59.000000,59.000000,57.000000,57.000000
mean,1.037034,8.600448,0.017224,7.462017,0.011186,6.251632,0.007211
std,0.298511,2.092962,0.004974,1.889636,0.003355,1.662180,0.002111
min,0.571000,4.586000,0.004000,3.749000,0.002000,2.564000,0.002000
25%,0.796500,7.450250,0.014250,6.264500,0.009000,5.204000,0.006000
50%,1.007000,8.662000,0.017000,7.504000,0.011000,6.301000,0.007000
75%,1.255500,9.915750,0.021000,8.887500,0.014000,7.486000,0.009000
max,1.653000,13.071000,0.027000,11.728000,0.018000,10.080000,0.011000


`describe()` gives us a summary of the dataframe. The first row, `count`, tells us the number of rows in each column that contain a value. We can see from this that 2 rows are missing data in the `mag_B` and `err_B` columns, and 3 are missing data in the `mag_I` and `err_I` columns. That's OK for now, we'll keep those rows in. 

(ex-check-missing-data)=
```{admonition} Exercise: Checking which stars are missing data
:class: practice

If you haven't already, download the [optical_data.csv](./data/optical_data.csv) file and save it in your `data` folder.

Read the data into a dataframe and use `head()`, `shape` and `describe()` to check your dataframe. 

Some of the stars are missing some data. Check which stars are missing data in any of the columns. If you need reminding how to check for missing data you can look back at the [cleaning data](https://moodle.bath.ac.uk/pluginfile.php/2458246/mod_resource/content/1/html/02_cleaning_data.html) section of C3.

[solution](soln_c4_missing_data.ipynb)
```

There are two other datafiles we're going to use: [gaia_distances.csv](./data/gaia_distances.csv) and [reddenings.csv](./data/reddenings.csv). 

The *Gaia* file contains parallax measurements in units of milliarcseconds (mas, or $10^{-3} \text{ arcseconds}$), for each star. 

The reddenings file contains the extinction, $A_V$, and reddening, $E(B-V)$, at the position of each star. These are both given in magnitudes.

```{admonition} Exercise: Read in the *Gaia* data and the reddening data
:class: practice
Download the [gaia_distances.csv](./data/gaia_distances.csv) and [reddenings.csv](./data/reddenings.csv) files and save them in your `data` folder. 

Read these files into dataframes and use `head()`, `shape` and `describe()` to check your them. 

For each of the three dataframes (optical data, *Gaia* data, reddenings) write some information about what the file contains (i.e. column names, units, number of objects, any missing data) in a markdown cell. 
```


If everything has worked correctly, your `gaia_df` dataframe should have 2 columns and 67 rows, and your `reddenings_df` dataframe should have 3 columns and 54 rows. Neither should have any `NaN` values. Your three dataframes should look like these:

In [6]:
gaia_df = pd.read_csv("./data/gaia_distances.csv")
reddenings_df = pd.read_csv("./data/reddenings.csv")

In [7]:
optical_df.head()

,Star_ID,logP,mag_B,err_B,mag_V,err_V,mag_I,err_I
0,RT Aur,0.571,6.120,0.017,5.487,0.011,4.822,0.006
1,QZ Nor,0.578,9.782,0.007,8.875,0.004,7.871,0.003
2,SU Cyg,0.585,7.493,0.015,6.890,0.011,6.208,0.007
3,Y Lac,0.636,9.921,0.016,9.163,0.011,8.312,0.007
4,T Vul,0.647,6.444,0.014,5.772,0.009,5.087,0.006


In [8]:
gaia_df.head()

,Star_ID,parallax_mas
0,XX Cen,0.564
1,T Mon,0.733
2,TW Nor,0.362
3,CV Mon,0.602
4,RY Sco,0.754


In [9]:
reddenings_df.head()

,Star_ID,E_B_V,A_V
0,RT Aur,0.1844,0.5717
1,QZ Nor,1.2364,3.8329
2,SU Cyg,0.9989,3.0967
3,Y Lac,0.3880,1.2028
4,T Vul,0.1702,0.5278


## Combining the dataframes

You may have noticed that while the `Star_ID` column exists in all the dataframes, the stars aren't in the same order. This is going to make it difficult to, for example, make a plot of `parallax` against `mag_V`. The star in row 1 in `gaia_df` is not the same star as row 1 in `optical_df`. What we need to do is combine all three of our dataframes into one big one. 

We can combine dataframes using **`pd.merge()`**

We'll start by combining `gaia_df` and `optical_df` with `pd.merge()`. You can find the documentation for `pd.merge()` [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

`merge()` takes two dataframes as input. One is the `left` dataframe and one is the `right` dataframe. We also need to tell it which column the two dataframes have in common - in this case it's the `Star_ID` column. This is defined by the `on` keyword The values in this column don't need to be in the same order, `pandas` can figure that out. 

The final thing we need to provide is how we want the data to be merged. This is the `how` keyword. 

There are several options for `how`:
* `left` - Uses the left dataframe as a reference, looking for matches in the right dataframe.
* `right` - The opposite of `left`
* `outer` - Keeps all the rows from both dataframes, if it can't find a match fills with `NaN`s
* `inner` - Keeps the rows where the values in the `on` column appear in both dataframes.

(There is a 5th option - `cross` which gives the "cartesian product" of the two dataframes. You can test this out if you want, but you won't need it for anything we're doing here.)

To see how the different `how` options work we're going to look quickly at two new dataframes. Copy the following code into a new cell in your notebook to create the `df1` and `df2` dataframes:


In [68]:
df1 = pd.DataFrame({'name': ['Alice', 'Bob', 'Clive', 'Doris',],
                        'colour': ['red', 'orange', 'yellow', 'green']})
                        
df2 = pd.DataFrame({'name': ['Ethel', 'Frank', 'Doris', 'Steve'],
                        'animal': ['dog', 'cat', 'llama', 'fish']})



Take a look at each dataframe. We'll use `df1` as the left and `df2` as the right. 

In [67]:
df1

,name,colour
0,Alice,red
1,Bob,orange
2,Clive,yellow
3,Doris,green


In [53]:
df2

,name,animal
0,Ethel,dog
1,Frank,cat
2,Doris,llama
3,Steve,fish


```{image} images/merging_1.png
:alt: visual representation of the two dataframes
:width: 800px
:align: center
```

```{admonition} What's actually happening? 
:class: myinfo
You'll see that I've included some additional figures in this section that show how the different merge options work. You won't get the coloured tables or the circles when you run the code. These are included to help you visualise what's happening in the different types of merge. 
```

Both have a column called `name`, so we'll match on that column for now.

### `left` merge
First we'll try a `left` merge - where it uses the values in the `name` column in `df1` as a reference and tries to find a match in `df2`:

In [54]:
pd.merge(left=df1, right=df2, on='name', how='left')

,name,colour,animal
0,Alice,red,NaN
1,Bob,orange,NaN
2,Clive,yellow,NaN
3,Doris,green,llama


```{image} images/merging_2.png
:alt: visual representation of the left merge
:width: 800px
:align: center
```

We can see that the only one it could find a match for was the last row, where `name=='Doris'`. It's kept the information from `df1` and filled in `NaN`s in the `animal` column for the rows that didn't have a match in `df2`. 

### `right` merge
If we switch to `how='right'` we get a different result to the `left` merge:

In [55]:
pd.merge(left=df1, right=df2, on='name', how='right')

,name,colour,animal
0,Ethel,NaN,dog
1,Frank,NaN,cat
2,Doris,green,llama
3,Steve,NaN,fish


```{image} images/merging_3.png
:alt: visual representation of the right merge
:width: 800px
:align: center
```

Here it's kept the information from `df2` and filled in `NaN`s in the `colour` column for rows that didn't have a match. 

The options that are going to be more useful for us are `outer` and `inner`. 

### `inner` merge
When we merge `df1` and `df2` with `how='inner'` we only get the rows that occur in **both** dataframes:

In [58]:
pd.merge(left=df1, right=df2, on='name', how='inner')

,name,colour,animal
0,Doris,green,llama


```{image} images/merging_4.png
:alt: visual representation of the inner merge
:width: 800px
:align: center
```

The inner merge is useful sometimes - we might only want to keep rows where we have all the data available.

### `outer` merge
The option we'll be using today is `how='outer'`:

In [59]:
pd.merge(left=df1, right=df2, on='name', how='outer')

,name,colour,animal
0,Alice,red,NaN
1,Bob,orange,NaN
2,Clive,yellow,NaN
3,Doris,green,llama
4,Ethel,NaN,dog
5,Frank,NaN,cat
6,Steve,NaN,fish


```{image} images/merging_5.png
:alt: visual representation of the outer merge
:width: 800px
:align: center
```

This keeps **all** the rows, and fills in missing values with `NaN`s. This is what we want.

Now we know what `merge` is doing we can merge the `gaia_df` and `optical_df` dataframes into one big one. We'll call the new dataframe `cepheids_df`.

In [60]:
cepheids_df = pd.merge(left=gaia_df, right=optical_df, on='Star_ID', how='outer')

In [61]:
cepheids_df

,Star_ID,parallax_mas,logP,mag_B,err_B,mag_V,err_V,mag_I,err_I
0,XX Cen,0.564,1.040,8.882,0.019,7.855,0.012,6.754,0.008
1,T Mon,0.733,1.432,7.436,0.022,6.187,0.014,5.005,0.010
2,TW Nor,0.362,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CV Mon,0.602,0.731,11.681,0.015,10.314,0.010,8.653,0.006
4,RY Sco,0.754,1.308,9.568,0.018,8.037,0.012,6.271,0.008
...,...,...,...,...,...,...,...,...,...
62,delta Cep,3.556,0.730,4.684,0.018,3.990,0.012,NaN,NaN
63,U Aql,1.748,0.847,7.536,0.016,6.457,0.011,5.279,0.007
64,eta Aql,3.674,0.856,4.744,0.017,3.918,0.011,3.036,0.007
65,RT Aur,1.841,0.571,6.120,0.017,5.487,0.011,4.822,0.006


We now have a new dataframe with 67 rows and 9 columns - the same number of rows that we had in the larger of the two dataframes (`gaia_df`) but now with all the columns from both dataframes. 

But `optical_df` only had 59 rows, so there must be some missing data. We should check...

In [62]:
cepheids_df[cepheids_df.isna().any(axis=1)]

,Star_ID,parallax_mas,logP,mag_B,err_B,mag_V,err_V,mag_I,err_I
2,TW Nor,0.362,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,V340 Nor,0.490,1.053,NaN,NaN,8.407,0.005,NaN,NaN
10,GY Sge,0.346,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,DL Cas,0.581,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,CE Cas B,0.331,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,V367 Sct,0.470,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,CF Cas,0.315,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,CE Cas A,0.331,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,S Vul,0.231,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,delta Cep,3.556,0.730,4.684,0.018,3.990,0.012,NaN,NaN


So we've still got the two stars that were just missing some of the photometric data in `optical_df` -- `delta Cep` and `TW Nor`, but now we've got some extra stars that don't have any photometric data. We'll keep them all in for now and tidy things up once we've finished merging.

(ex-merging)=
```{admonition} Exercise: Add in the reddening columns
:class: practice

Use `pd.merge()` again to add the columns from `reddening_df` to the `cepheids_df` dataframe. Check what the new dataframe looks like and if there is any missing data.

**Hint:** You can run `merge` so that the resulting dataframe overwrites one of your existing ones. You don't have to create a new dataframe.

[solution](soln_merging.ipynb)
```



You should now have a dataframe called `cepheids_df` that looks something like this:


In [63]:
cepheids_df = pd.merge(left=cepheids_df, right=reddenings_df, on='Star_ID', how='outer')


In [64]:
cepheids_df

,Star_ID,parallax_mas,logP,mag_B,err_B,mag_V,err_V,mag_I,err_I,E_B_V,A_V
0,XX Cen,0.564,1.040,8.882,0.019,7.855,0.012,6.754,0.008,0.5223,1.6191
1,T Mon,0.733,1.432,7.436,0.022,6.187,0.014,5.005,0.010,0.5997,1.8590
2,TW Nor,0.362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CV Mon,0.602,0.731,11.681,0.015,10.314,0.010,8.653,0.006,1.4373,4.4556
4,RY Sco,0.754,1.308,9.568,0.018,8.037,0.012,6.271,0.008,1.2228,3.7907
...,...,...,...,...,...,...,...,...,...,...,...
62,delta Cep,3.556,0.730,4.684,0.018,3.990,0.012,NaN,NaN,1.4914,4.6233
63,U Aql,1.748,0.847,7.536,0.016,6.457,0.011,5.279,0.007,0.3334,1.0337
64,eta Aql,3.674,0.856,4.744,0.017,3.918,0.011,3.036,0.007,0.1936,0.6001
65,RT Aur,1.841,0.571,6.120,0.017,5.487,0.011,4.822,0.006,0.1844,0.5717


## Saving the data to a file

Now we have our nicely matched dataframe, we don't want to have to do all these steps every time we want to analyse the data. We can save the dataframe to a csv file using `pd.to_csv()`.

This works in a very similar way to `pd.read_csv()`. There are a few arguments that we won't have used for `read_csv` that we'll want to use now.


```{admonition} Slow down!
:class: warning
The following text is an explanation of the options. Don't try to run each line separately. We'll be putting it together at the end. 
```



Firstly, `index`:
```py 
index=False
```
`index` is the number you see in bold in the leftmost column when you display a dataframe. It's handy when you want to select a row while you're working with the data in `pandas`, but we don't need to save it to the file. **Note:** There may be times that you *do* want to write the index to the file, like if you wrote a script that had the index values it needed to use hard-coded, but generally you don't need them in the output file.

Second, `float_format`:
```py
float_format='%.4f'
```
This tells `to_csv` how to format the numbers in the output file. `float_format='%.4f'` means 4 decimal places. If you don't set anything for `float_format` it will print *all* the decimal places. Sometimes that's what you want it to do, but it's more likely that you want to preserve the precision you have in your dataframe without all the noise that comes from your computer rounding `floats`.

Next, `na_rep`:
```py
na_rep = 'NaN'
```
This tells it to print missing data as `NaN` in the file. If you don't set it it will just print nothing (which can be OK...) but it's better to print something so when you come back to the file later you know that it's actually missing data, not just that you've accidentally deleted something from the file. 

Let's save `cepheids_df` to a file called `big_cepheids_table.csv`. Save it somewhere sensible! (For example, your `data` folder...)

Finally, `sep` is the equivalent of `delimiter` in `read_csv`. The default is `sep=','` (i.e. comma separated variables) but you can change it to whatever you want. I find it easiest to leave it as the default so I don't have to remember what I used when I'm reading the file back in.

In [65]:
cepheids_df.to_csv('./data/big_cepheids_table.csv', index=False, float_format='%.4f', na_rep='NaN')

Now we have our data safe and sound we can go and have a cup of tea and come back to this later.

## Key Points
- Use `pd.merge()` to combine dataframes
- There are different ways `how` you can join your dataframes:
  - `outer` keeps all the rows and fills missing data with `NaN`s
  - `inner` only keeps the rows where the reference value appears in both frames
  - `left` and `right` keep the rows from the reference frame and fill the missing data with `NaN`s
- Save a dataframe to a file with `pd.to_csv()`